In [1]:
import pandas as pd 
import numpy as np
import os
import pyspark 
from pyspark.sql import SparkSession

In [2]:
#replace with correct directory where data files are located 
os.chdir(r'C:\Users\Nicho\gc-march-2022\data')

In [3]:
da = pd.read_csv('Indeed_DataAnalyst_2022.csv')
de = pd.read_csv('Indeed_DataEngineer_2022.csv')
ds = pd.read_csv('Indeed_DataScience_2022.csv')

In [4]:
da.info()
de.info()
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1155 non-null   int64 
 1   title        1155 non-null   object
 2   company      1153 non-null   object
 3   salary       388 non-null    object
 4   description  1153 non-null   object
 5   location     1153 non-null   object
dtypes: int64(1), object(5)
memory usage: 54.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4504 entries, 0 to 4503
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   4504 non-null   int64 
 1   title        4504 non-null   object
 2   company      4504 non-null   object
 3   salary       1256 non-null   object
 4   description  4504 non-null   object
 5   location     4504 non-null   object
dtypes: int64(1), object(5)
memory usage: 211.2+ KB
<class 'pandas.core.frame.Dat

There is a lot more rows in the engineering data than the other two sites. Only the salary column contains null values.

In [5]:
indeed_df = pd.concat([da,de,ds])
indeed_df.info()
indeed_df = indeed_df[['title','company','salary','description','location']]
indeed_df.drop_duplicates(inplace=True)
indeed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6812 entries, 0 to 1152
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   6812 non-null   int64 
 1   title        6812 non-null   object
 2   company      6810 non-null   object
 3   salary       1985 non-null   object
 4   description  6810 non-null   object
 5   location     6810 non-null   object
dtypes: int64(1), object(5)
memory usage: 372.5+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2210 entries, 0 to 1121
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2210 non-null   object
 1   company      2208 non-null   object
 2   salary       725 non-null    object
 3   description  2208 non-null   object
 4   location     2208 non-null   object
dtypes: object(5)
memory usage: 103.6+ KB


In [6]:
indeed_df['salary'] = indeed_df['salary'].str.lower()
indeed_df['description'] = indeed_df['description'].str.lower()
indeed_df['location'] = indeed_df['location'].str.lower()

In [7]:
indeed_df.head(5)

,title,company,salary,description,location
0,Data Science Analyst- Marketing,Indeed,"$100,000 - $140,000 a year",work across a wide range of data engineering t...,remote in texas+1 location
1,Data Analyst/Statistician,US Department of Veterans Affairs,"$51,980 - $103,518 a year",create processes for assessing quality and acc...,"hybrid remote in ann arbor, mi 48105"
2,Departmental Analyst 9-P11 - Data Analytics,State of Michigan,NaN,"and reporting, including biannual reports to t...","dimondale, mi"
3,CRM Data Analyst,Zeiss Industrial Quality Solutions,"$60,000 - $70,000 a year",responsible for maintaining sap/crm business p...,"wixom, mi 48393+1 location"
4,Data Analyst,Winwire Technologies,NaN,providing resource to add/delete/amend data wi...,remote


In [8]:
import re
indeed_df['salary'] = indeed_df['salary'].str.replace(',','')
indeed_df['salary'] = indeed_df['salary'].str.replace('-',' ')
indeed_df['salary'] = indeed_df['salary'].str.replace('$',' ')
indeed_df['salary'] = indeed_df['salary'].str.replace("[a-z]","")   
indeed_df['jobid'] = str(indeed_df['title']) + str(indeed_df['company']) + str(indeed_df['company'])


C:\Users\Nicho\AppData\Local\Temp/ipykernel_24860/3954269124.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  indeed_df['salary'] = indeed_df['salary'].str.replace('$',' ')
C:\Users\Nicho\AppData\Local\Temp/ipykernel_24860/3954269124.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['salary'] = indeed_df['salary'].str.replace("[a-z]","")


In [9]:
income_split = indeed_df['salary'].str.split(expand = True)
indeed_df['salary_lower_range'] =  income_split[0]
indeed_df['salary_upper_range'] =  income_split[1]
indeed_df.drop('salary',axis = 1,inplace=True)

In [10]:

indeed_df['company'].dropna(axis=0,inplace=True)
indeed_df['salary_lower_range'].fillna(0,inplace=True)
indeed_df['salary_upper_range'].fillna(0,inplace=True)
indeed_df['salary_lower_range'] = indeed_df['salary_lower_range'].astype(float)
indeed_df['salary_upper_range']  = indeed_df['salary_upper_range'].astype(float)
hourly_salaries = indeed_df[indeed_df['salary_lower_range'] < 100]
indeed_df = indeed_df[indeed_df['salary_lower_range'] > 100]

In [11]:
hourly_salaries['salary_lower_range']  = (hourly_salaries['salary_lower_range'] * 40) * 52
hourly_salaries['salary_upper_range'] = (hourly_salaries['salary_upper_range'] * 40) * 52

In [12]:
indeed_df = pd.concat([hourly_salaries,indeed_df])

In [13]:
indeed_df['is_remote'] = np.where(indeed_df['description'].str.contains('remote'),1,0)
indeed_df['location'] = indeed_df['location'].str.replace('remote','')
indeed_df['location'] = indeed_df['location'].str.replace('in','')

In [14]:
indeed_df.groupby('is_remote').count()

,title,company,description,location,jobid,salary_lower_range,salary_upper_range
is_remote,,,,,,,
0,2189,2189,2189,2189,2189,2189,2189
1,21,19,19,19,21,21,21


In [15]:
indeed_df['zipcode'] = indeed_df['location'].str.replace(r'\D+','')
indeed_df['zipcode'] = np.where(indeed_df['zipcode'] == '',0,indeed_df['zipcode'])
indeed_df['zipcode'] = indeed_df['zipcode'].fillna(0)
indeed_df['zipcode'] = indeed_df['zipcode'].astype('int')
indeed_df['zipcode'] = np.where(indeed_df['zipcode'] < 1000,0,indeed_df['zipcode'])
indeed_df.drop_duplicates(inplace = True)

C:\Users\Nicho\AppData\Local\Temp/ipykernel_24860/730417472.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['zipcode'] = indeed_df['location'].str.replace(r'\D+','')


In [16]:
indeed_df['location'] = indeed_df['location'].str.replace("[0-9]",'')

C:\Users\Nicho\AppData\Local\Temp/ipykernel_24860/1525615495.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['location'] = indeed_df['location'].str.replace("[0-9]",'')


In [17]:
indeed_df['location'] = indeed_df['location'].str.strip()
indeed_df['location'] = indeed_df['location'].str.split(',')

In [18]:
indeed_df['city'] = indeed_df['location'].str[0]
indeed_df['city'] = indeed_df['city'].str.replace("[0-9+()-]",' ')
indeed_df['state'] = indeed_df['location'].str[1]
indeed_df['state'] = indeed_df['state'].str.split()
indeed_df['state'] = indeed_df['state'].str[0]
indeed_df['state'] = indeed_df['state'].str.replace("[0-9+()-]",'')

C:\Users\Nicho\AppData\Local\Temp/ipykernel_24860/3846399863.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['city'] = indeed_df['city'].str.replace("[0-9+()-]",' ')
C:\Users\Nicho\AppData\Local\Temp/ipykernel_24860/3846399863.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['state'] = indeed_df['state'].str.replace("[0-9+()-]",'')


In [19]:
indeed_df['city'] = np.where(indeed_df['city'].str.contains('location'),'Remote',indeed_df['city'])
indeed_df.groupby('state').count()
indeed_df.groupby('city').count()           
indeed_df.drop('location',axis = 1,inplace=True)

In [20]:
indeed_df

,title,company,description,jobid,salary_lower_range,salary_upper_range,is_remote,zipcode,city,state
2,Departmental Analyst 9-P11 - Data Analytics,State of Michigan,"and reporting, including biannual reports to t...",0 Data Science Analyst...,0.0,0.0,0,0,dimondale,mi
4,Data Analyst,Winwire Technologies,providing resource to add/delete/amend data wi...,0 Data Science Analyst...,0.0,0.0,0,0,,NaN
5,Junior Data Analyst - DAL22-033349999999,"Evergreen Technologies, LLC.","experience with data wrangling, data cleaning ...",0 Data Science Analyst...,0.0,0.0,0,0,detroit,mi
6,Junior Data Analyst,Teamson,at least two years of experience as an analyst...,0 Data Science Analyst...,0.0,0.0,0,0,,NaN
8,"Senior Data Analyst, IT Remote",U.S. Medical Management,perform data cleansing to identify erroneous d...,0 Data Science Analyst...,0.0,0.0,0,48084,troy,mi
...,...,...,...,...,...,...,...,...,...,...
1088,Software Engineering Manager,CyberCoders,"lead, manage, mentor and develop teams of soft...",0 Data Science Analyst...,170000.0,200000.0,0,90066,los angeles,ca
1095,Bi Architect,CyberCoders,this data-first strategy has enabled data and ...,0 Data Science Analyst...,150000.0,200000.0,0,72761,siloam sprgs,ar
1100,Software Engineering Manager,CyberCoders,"to meet business and consumer needs, we utiliz...",0 Data Science Analyst...,170000.0,200000.0,0,90066,los angeles,ca
1102,Director of Regional Insights and Analytics,Michael Page,"academic background in statistics, data scienc...",0 Data Science Analyst...,150000.0,160000.0,1,0,jersey city,nj
